In [ ]:
import numpy as np
import pandas as pd 
import sys, os, psutil, io
import gzip
import time 
import warnings

from pybedtools import BedTool
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [ ]:
#data_path='/Users/shu/pollard_lab/atrt/'
#annotated_del=pd.read_csv(data_path + '20230802_del_test_annotated.txt', sep='\t')

synteny_condensed=pd.read_csv('/pollard/home/shzhang/data/synteny/hg38_mm10_condensed.txt', sep='\t')

In [ ]:
synteny_condensed

In [ ]:
data_path='/pollard/data/projects/kgjoni/CBTN_collab/CBTN_data/'
#simple_df=pd.read_csv(data_path + 'simple-variants_230620/simple_variants_all', sep='\t')
sv_df=pd.read_csv(data_path + 'structural-variants_230620/SVs_all', sep='\t')
sv_intra=sv_df[sv_df.SVTYPE != 'BND']
sv_intra = sv_intra.reset_index(drop=True)
sv_intra

In [ ]:
#merge with chr len information
chrom_len=pd.read_csv('/pollard/home/shzhang/akita_variant_scoring/data/chrom_lengths_hg38', sep='\t', header=None)
chrom_len[0]='chr' + chrom_len[0]
sv_intra=sv_intra.merge(chrom_len, right_on=0, left_on='CHROM', how='left')


In [ ]:
chrom_len

In [ ]:
#ALSO ADD BOUNDARIES FOR 1MB IN SV_INTRA


sv_intra = sv_intra.rename(columns={1: 'chr_len'})
sv_intra['half_len']=abs(sv_intra['END']-sv_intra['POS'])/2
sv_intra['half_len']=np.minimum(sv_intra['half_len'], 5e5)

sv_intra['POS_1MB']=np.floor(np.maximum(0, sv_intra['POS']-(5e5-sv_intra['half_len']) ))
sv_intra['END_1MB']=np.floor(np.minimum(sv_intra['chr_len'], sv_intra['END']+ (5e5-sv_intra['half_len']) ))
sv_intra['postoend_1mb']=sv_intra['END_1MB']-sv_intra['POS_1MB']
sv_intra['postoend']=sv_intra['END']-sv_intra['POS']

sv_intra=sv_intra[sv_intra.postoend<7e5]
sv_intra = sv_intra.reset_index(drop=True)


In [ ]:
sv_intra

In [ ]:
242893384.0-241893384.0

In [ ]:
pd.set_option('display.max_colwidth', 40)
print(sv_intra[107:108]['INFO'])

In [ ]:
len(np.unique(sv_intra.ID))

In [ ]:
#check that lengths of seqs and alignments are correct  
synteny_condensed['hg_dash_n']=synteny_condensed['hg_seq'].str.count('-')
synteny_condensed['mm_dash_n']=synteny_condensed['mm_seq'].str.count('-')

synteny_condensed['hg_percent']=1-(synteny_condensed['hg_dash_n']/synteny_condensed['total_align_len'])
synteny_condensed['mm_percent']=1-(synteny_condensed['mm_dash_n']/synteny_condensed['total_align_len'])


print(((synteny_condensed['hg_len'] + synteny_condensed['hg_dash_n']) == synteny_condensed['total_align_len']).all())
print(((synteny_condensed['mm_len'] + synteny_condensed['mm_dash_n']) == synteny_condensed['total_align_len']).all())

In [ ]:
np.median(synteny_condensed['hg_percent'])

In [ ]:
sv_intra


In [ ]:
variants=sv_intra
variants=variants[['CHROM', 'POS', 'END', 'ID']]
variants['END']=variants['END'].astype(int)
print(len(variants))
variants.head()

In [ ]:
variants_1mb=sv_intra
variants_1mb=variants_1mb[['CHROM', 'POS_1MB', 'END_1MB', 'ID']]
variants_1mb['POS_1MB']=variants_1mb['POS_1MB'].astype(int)
variants_1mb['END_1MB']=variants_1mb['END_1MB'].astype(int)
print(len(variants_1mb))
variants_1mb.head()

In [ ]:
variant_BED = BedTool.from_dataframe(variants)
variant_1mb_BED = BedTool.from_dataframe(variants_1mb)

In [ ]:
synteny_BED = BedTool.from_dataframe(synteny_condensed)


In [ ]:
#reports only SVs where there is overlap with  gene
#-F ensures that at least 0.5 of gene has overlap with SV 
#use loj in this case, because many SVs overlap multiple syntenic regions  
variant_synt=variant_BED.intersect(synteny_BED, loj=True, wo=True).to_dataframe(header=None)

variant_1mb_synt=variant_1mb_BED.intersect(synteny_BED, loj=True, wo=True).to_dataframe(header=None)




In [ ]:
variant_synt.columns = list(variants.columns) + list(synteny_condensed.columns)
variant_synt = variant_synt[variant_synt['hg_chr'] != '.']
print(len(variant_synt))
variant_synt.head()

In [ ]:
variant_1mb_synt.columns = list(variants_1mb.columns) + list(synteny_condensed.columns)
variant_1mb_synt = variant_1mb_synt[variant_1mb_synt['hg_chr'] != '.']
print(len(variant_1mb_synt))
variant_1mb_synt.head()

In [ ]:
def find_index(hg_seq, idx):
    count_letters = 0  # To count the letters encountered in the combined string
    idx_new = 0  # Corresponding index in combined string
    
    for char in hg_seq:
        if count_letters == (idx+1):
            break
        if char.isalpha():
            count_letters += 1
        idx_new += 1
    
    return idx_new

#given two aligned strings with start and end idx
#count up amount of same characters within the alignment subset 
def count_str(hg_seq, mm_seq, start_idx, end_idx):
    hg_subset = hg_seq[start_idx:end_idx]
    mm_subset = mm_seq[start_idx:end_idx]

    count_same = sum(1 for a, b in zip(hg_subset, mm_subset) if a == b)
    prop=count_same/(end_idx-start_idx)
    
    
    return(prop)

In [ ]:
#TEST THAT THIS WORKS

# Example strings
hg_seq = "a-t-t--gtc-aa"
idx = 3  # 0-indexed value

# Find the corresponding index in the combined string
corresponding_index = find_index(hg_seq, idx)
print("Corresponding index:", corresponding_index)

In [ ]:
start=time.time()
for i in sv_intra.index:
   
    if i % 100 == 0:
        print('index ', i)
    
    
    #THIS ONE FOR 1MB SCORE 
    try:
        #print(i)
        variant=sv_intra.loc[i]
        ID=variant.ID
        int_df=variant_1mb_synt[variant_1mb_synt['ID'] == ID]

        #print(len(int_df))
    
    
        if len(int_df)>0:


            int_df['start_idx']=(np.maximum(int_df['POS_1MB'], int_df['hg_start'])) - int_df['hg_start'] 
            int_df['end_idx']=(np.minimum(int_df['END_1MB'], int_df['hg_end'])) - int_df['hg_start'] 
            int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
            #get alignment indices
            int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
            int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
            int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
            int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'],
                                                                      row['start_align_idx'], row['end_align_idx']), axis=1)


            #proportion of SV that covers a syntenic region  
            sv_intra.loc[i, 'synt_prop_overlap_1MB'] = int(np.sum(int_df['n_overlap'])) /abs(sv_intra.loc[i, 'postoend_1mb'])
            #weighted proportion of SV in syntenic region, and how similar those regions are 
            sv_intra.loc[i, 'align_synt_prop_1MB'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(sv_intra.loc[i, 'postoend_1mb'])



        else:
            sv_intra.loc[i, 'synt_prop_overlap_1MB'] = 0
            sv_intra.loc[i, 'align_synt_prop_1MB'] = 0
            
    except:
        print('error in index', i)
        
        
    #THIS ONE FOR NORMAL SV LENGTH 
    try:
        #print(i)
        variant=sv_intra.loc[i]
        ID=variant.ID
        int_df=variant_synt[variant_synt['ID'] == ID]
        #print(len(int_df))
    
    
        if len(int_df)>0:


            int_df['start_idx']=(np.maximum(int_df['POS'], int_df['hg_start'])) - int_df['hg_start'] 
            int_df['end_idx']=(np.minimum(int_df['END'], int_df['hg_end'])) - int_df['hg_start'] 
            int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
            #get alignment indices
            int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
            int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
            int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
            int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'],
                                                                      row['start_align_idx'], row['end_align_idx']), axis=1)


            #proportion of SV that covers a syntenic region  
            sv_intra.loc[i, 'synt_prop_overlap'] = int(np.sum(int_df['n_overlap'])) /abs(sv_intra.loc[i, 'postoend'])
            #weighted proportion of SV in syntenic region, and how similar those regions are 
            sv_intra.loc[i, 'align_synt_prop'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(sv_intra.loc[i, 'postoend'])



        else:
            sv_intra.loc[i, 'synt_prop_overlap'] = 0
            sv_intra.loc[i, 'align_synt_prop'] = 0
            
    except:
        print('error 1mb in index', i)
        
        
end=time.time()
print(f"Time taken: {end-start} seconds")
    

In [ ]:
sv_intra.to_csv('/pollard/home/shzhang/20231023_intra_sv_synteny_combined.txt', sep='\t', index=False)  

In [ ]:
sv_intra[1790:1800]

In [ ]:
#THIS ONE IF ONLY THE NORMAL SV SCORES 
start=time.time()
for i in sv_intra.index:
   
    if i % 100 == 0:
        print('index ', i)
    
    try:
        #print(i)
        variant=sv_intra.loc[i]
        ID=variant.ID
        int_df=variant_synt[variant_synt['ID'] == ID]
        #print(len(int_df))
    
    
        if len(int_df)>0:


            int_df['start_idx']=(np.maximum(int_df['POS'], int_df['hg_start'])) - int_df['hg_start'] 
            int_df['end_idx']=(np.minimum(int_df['END'], int_df['hg_end'])) - int_df['hg_start'] 
            int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
            #get alignment indices
            int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
            int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
            int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
            int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'],

            #proportion of SV that covers a syntenic region  
            sv_intra.loc[i, 'synt_prop_overlap'] = int(np.sum(int_df['n_overlap'])) /abs(sv_intra.loc[i, 'SVLEN'])
            #weighted proportion of SV in syntenic region, and how similar those regions are 
            sv_intra.loc[i, 'align_synt_prop'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(sv_intra.loc[i, 'SVLEN'])



        else:
            sv_intra.loc[i, 'synt_prop_overlap'] = 0
            sv_intra.loc[i, 'align_synt_prop'] = 0
            
    except:
        print('error in index', i)
        
        
end=time.time()
print(f"Time taken: {end-start} seconds")
    

In [ ]:
sv_intra.to_csv('/pollard/home/shzhang/20231023_intra_sv_synteny_combined.txt', sep='\t', index=False)  

In [ ]:
test1=pd.read_csv('/pollard/home/shzhang/20230912_intra_sv_synteny.txt', sep='\t')
test1

In [ ]:
for i in sv_intra.index:

    #print(i)
    variant=sv_intra.loc[i]
    ID=variant.ID
    int_df=variant_synt[variant_synt['ID'] == ID]
    #print(len(int_df))
    
    
    if len(int_df)>0:


        int_df['start_idx']=(np.maximum(int_df['POS'], int_df['hg_start'])) - int_df['hg_start'] 
        int_df['end_idx']=(np.minimum(int_df['END'], int_df['hg_end'])) - int_df['hg_start'] 
        int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
        #get alignment indices
        int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
        int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
        int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
        int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'],
                                                                  row['start_align_idx'], row['end_align_idx']), axis=1)

        #print(int_df)


        #proportion of SV that covers a syntenic region  
        sv_intra.loc[i, 'synt_prop_overlap'] = int(np.sum(int_df['n_overlap'])) /abs(sv_intra.loc[i, 'SVLEN'])
        #weighted proportion of SV in syntenic region, and how similar those regions are 
        sv_intra.loc[i, 'align_synt_prop'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(sv_intra.loc[i, 'SVLEN'])



    else:
        sv_intra.loc[i, 'synt_prop_overlap'] = 0
        sv_intra.loc[i, 'align_synt_prop'] = 0
        
        

In [ ]:
pd.options.display.max_columns = None
sv_intra.head()


In [ ]:
#look at distritbution of synteny scores 

plt.scatter(annotated_del['synt_prop_overlap'], annotated_del['align_synt_prop'], alpha=0.5)
plt.show()

In [ ]:
#cases to consider
#variant in no syntenic region 
# variant is fully in one syntenic region
# variant partially in one syntenic region
# variant in two or more syntenic regions 